In [ ]:
# trnsl.1.1.20191111T143250Z.0b93f423ea6fd88d.318f442d3d8cae900cb4404cef26667699fd5768

In [ ]:
import sys
try:
    sys.path.insert(0, "/usr/lib/python3.7/site-packages")
except FileNotFoundError:
    pass

import requests
import pandas as pd
import numpy as nm

### to see process info

In [ ]:
class nice_out:
    def __init__(self, L):
        self.L = L
        self.counter = 0
        self.per = 0
    def update(self):
        if(self.per == (self.counter*1000)//self.L):
            print("\r" + str(self.per/10) + "%" + " " + "#"*(self.per//20) + "_"*(50 - self.per//20), end = "")
            self.per += 1
        self.counter += 1
    def clear(self):
        print("\r" + " "*100, end = "\r")

## Functions

In [ ]:
# request.text = {"code":200,"lang":"en-ru","text":["OUTPUT"]}
def parse_request(request):
    text = request.text
    text = text[text.find("text") + 8: len(text) - 3].replace('\\', '')
    return text

def translate(text, key):
    URL = 'https://translate.yandex.net/api/v1.5/tr.json/translate?'
    translator = {'key': key, 'text': text, 'lang': "en-ru"}
    return parse_request(requests.post(URL, translator))
    
def str2list(x):
    return [y.strip() for y in x[2:-2].replace('"','').replace("'",'').split(',')]
def list2set(x):
    return set(x)

# Yandex translator

In [ ]:
#key - your key for Yandex translator
def translate_data(key, path_to_source, path_to_result="Rus_Yandex_TEST.csv"):
    df = pd.read_csv(path_to_source)[['Event','Xintent','Xemotion','Otheremotion']]
    cnt = nice_out(df.shape[0])
    for i in range(0, df.shape[0]):
        cnt.update()
        to_translate = (df.iloc[i]["Event"].replace(".", "") + ". " + 
                        df.iloc[i]["Xintent"].replace(".", "") + ". " + 
                        df.iloc[i]["Xemotion"].replace(".", "") + ". " + 
                        df.iloc[i]["Otheremotion"].replace(".", ""))
        translated = translate(to_translate, key)
        eve, xin, xre, oth = translated.split(".")
        
        df.iloc[i]["Event"]        = eve
        df.iloc[i]["Xintent"]      = xin.replace('\\', '').replace('.', '').strip()
        df.iloc[i]["Xemotion"]     = xre.replace('\\', '').replace('.', '').strip()
        df.iloc[i]["Otheremotion"] = oth.replace('\\', '').replace('.', '').strip()
    cnt.clear()
    df.to_csv(path_to_result, encoding="utf-8")
    
    print(df.head())

In [ ]:
URL = 'https://translate.yandex.net/api/v1.5/tr.json/translate?'
key = 'trnsl.1.1.20191109T120949Z.0b64f5bbabf5efcd.806496e582cf38c63bef5b7590f3fdf6ea939e2a'
lang = 'en-ru'

text1 = "The dog has a bone"
text2 = "This text has no misprints"
text3 = "I love C++"
text4 = '["excited", "eager", "interested"]'
print(translate(text1, key))
print(translate(text2, key))
print(translate(text3, key))
print(translate(text4, key))

## Russian Dataset

#### params:
#### path_to_source - way (and the name) to english file on your computer
#### path_to_result - way (and the name) to file where russian database will be saved

In [ ]:
translate_data(key, '/home/fomius2000/Documents/Python_proj/NLP/event2mind/test.csv')